# Memory

In [1]:
from secret_key import hugging_facehub_key
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hugging_facehub_key

In [2]:
from langchain import HuggingFaceHub
# Define LLM
repo_id = "google/flan-t5-large"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

C:\Users\Dell\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


# Chat Message History

In [3]:
from langchain.memory import ChatMessageHistory
from langchain import PromptTemplate, LLMChain

In [4]:
# Initialize chat message history
history = ChatMessageHistory()

# Add user and AI messages to the history
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [5]:
# Print the current messages in the history
print(history.messages)

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]


In [6]:
# Add another user message and print the updated history
history.add_user_message("Fine, what about you?")
print(history.messages)

[HumanMessage(content='hi!'), AIMessage(content='whats up?'), HumanMessage(content='Fine, what about you?')]


In [7]:
# Define a prompt template with a question variable
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [8]:
# Create an LLMChain with the prompt template and a language model
# Note: 'llm' should be initialized before using it here
llm_chain = LLMChain(prompt=prompt, llm=llm)

C:\Users\Dell\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
# Define a question and run the chain to get the model's response
question = "Which city does the company's headquarters for our international employees reside in?"
print(llm_chain.run(question))

C:\Users\Dell\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The company's headquarters for our international employees reside in New York City. New York City is a city. New York City is a city. So the answer is New York.


## Conversation Buffer Memory

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [11]:
conversation.predict(input="Hi, my name is SAM")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is SAM
AI:

> Finished chain.


'SAM: Hello, how can I help you?'

In [12]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is SAM
AI: SAM: Hello, how can I help you?
Human: What is 1+1?
AI:

> Finished chain.


'SAM: 1+1 is a number that is a combination of two numbers.'

In [13]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is SAM
AI: SAM: Hello, how can I help you?
Human: What is 1+1?
AI: SAM: 1+1 is a number that is a combination of two numbers.
Human: What is my name?
AI:

> Finished chain.


'SAM: My name is SAM.'

In [14]:
print(memory.buffer)

Human: Hi, my name is SAM
AI: SAM: Hello, how can I help you?
Human: What is 1+1?
AI: SAM: 1+1 is a number that is a combination of two numbers.
Human: What is my name?
AI: SAM: My name is SAM.


In [15]:
memory.load_memory_variables({})

{'history': 'Human: Hi, my name is SAM\nAI: SAM: Hello, how can I help you?\nHuman: What is 1+1?\nAI: SAM: 1+1 is a number that is a combination of two numbers.\nHuman: What is my name?\nAI: SAM: My name is SAM.'}

In [16]:
memory = ConversationBufferMemory()

In [17]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [18]:
print(memory.buffer)

Human: Hi
AI: What's up


In [19]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [20]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [21]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [22]:
from langchain.memory import ConversationBufferWindowMemory

In [23]:
memory = ConversationBufferWindowMemory(k=1)               

In [24]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [25]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [26]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [27]:
conversation.predict(input="Hi, my name is Andrew")

'Andrew: Hello, how can I help you?'

In [28]:
conversation.predict(input="What is 1+1?")

'Andrew: 1+1 is a number that is a combination of two numbers.'

In [29]:
conversation.predict(input="What is my name?")

'Andrew: Andrew is a fictional character.'

## ConversationTokenBufferMemory

In [30]:
#!pip install tiktoken

In [31]:
from langchain.memory import ConversationTokenBufferMemory

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

## ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
memory.load_memory_variables({})

## Knowledge Graph Memory

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain import HuggingFaceHub
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
memory = ConversationBufferWindowMemory(k=1)  

In [ ]:
repo_id = "google/flan-t5-large"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

In [ ]:
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

In [ ]:
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm)
)

In [ ]:
conversation_with_kg.predict(input="Hi there! I am Sam")

In [ ]:
conversation_with_kg.predict(input="My TV is broken and I need some customer assistance")

In [ ]:
conversation_with_kg.predict(input="My TV is LED sunsung")

In [ ]:
conversation_with_kg.predict(input="Yes it is and it is still under warranty. my warranty number is A512423")

In [ ]:
# nx.draw(conversation_with_kg.memory.kg, with_labels=True)
# plt.show()

print(conversation_with_kg.memory.kg)
print(conversation_with_kg.memory.kg.get_triples())

## Entity Memory

In [ ]:
from langchain.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain import HuggingFaceHub
from pprint import pprint

In [ ]:
ENTITY_MEMORY_CONVERSATION_TEMPLATE.template

In [ ]:
# Define LLM
repo_id = "google/flan-t5-large"  # The ID of the pre-trained model on Hugging Face
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

In [ ]:
memory = ConversationEntityMemory(llm=llm)

In [ ]:
# Initialize ConversationChain
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=memory
)

In [ ]:
conversation.predict(input="Hi I am Sam. My TV is broken but it is under warranty.")

In [ ]:
conversation.predict(input="How can I get it fixed. The warranty number is A512453")

In [ ]:
conversation.memory.entity_cache

In [ ]:
conversation.predict(input="Can you send the repair person call Dave to fix it?.")

In [ ]:
conversation.memory.entity_cache

In [ ]:
pprint(conversation.memory)